# Stochastic Swim Meet Simulator

In this notebook, I attempt to build a probabilistic simulator for the Missouri Boys High School State 2025 Swimming Championships using a **Markov-ish performance state** model & **Monte Carlo** simulation.

Here is my process:
1) Load in CSV data & formatting properly
2) Create variability in every swimmer's performance using a simple Markov performance state
3) Adaptive interdependence, where if one person on the team does good, the rest also do better (team momentum effect)
4) Do the Monte Carlo loop to estimate each team's win probabilities and distributions
5) Integer linear programming to optimize events for swimmer (2 individual events per swimmer, will add relay stuff later)

Summary: (1) Markov chain for each swimmer &rarr; (2) Monte Carlo sampling across many simulated meets &rarr; (3) Resulting outcome distribution


In [163]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import random

import matplotlib.pyplot as plt
import seaborn as sns

## 1) Load & prep data

Input CSV format (downloaded from the MSHSAA website):
- Empty column headers, except for event names
- Then columns have place, time, competitor name, school name which repeats horizontally

Output CSV format:
- `name`: swimmer name
- `team`: school name
- `event`: event name, (relays not done yet)
- `best_time`: best seed time in seconds this season
- `seed_rank`: rank/seed for that event (1 = top seed)

In [164]:
df = pd.read_csv('missouri_swim_data.csv', header=None)

# Extract event names from first row (skip empty columns)
events = []
event_cols = []
for i in range(0, len(df.columns)-1):
    if i < len(df.columns) and pd.notna(df.iloc[0, i+1]):
        event_name = df.iloc[0, i+1]
        events.append(event_name)
        event_cols.append(i)

In [165]:
events

['200 Medley Relay',
 '200 Freestyle',
 '200 Individual Medley',
 '50 Freestyle',
 '100 Butterfly',
 '100 Freestyle',
 '500 Freestyle',
 '200 Freestyle Relay',
 '100 Backstroke',
 '100 Breaststroke',
 '400 Freestyle Relay']

In [166]:
# Initialize list to store all swimmer records
all_records = []

# Process each event
for event_idx, (event_name, col_start) in enumerate(tqdm(zip(events, event_cols), total=len(events), desc="Processing Events")):
    # Determine if this is a relay event (check if there are swimmer names)
    is_relay = 'Relay' in event_name
    
    # Process each row (starting from row 1, skipping header row 0)
    for row_idx in tqdm(range(1, len(df)), leave=False, desc=f"Event {event_name}"):
        rank = df.iloc[row_idx, col_start - 1]
        time = df.iloc[row_idx, col_start]
        name_or_team = df.iloc[row_idx, col_start + 1]
        team = df.iloc[row_idx, col_start + 2] if not is_relay else None
        
        # Skip if rank or time is missing
        if pd.isna(rank) or pd.isna(time) or pd.isna(name_or_team):
            continue
        
        # Convert time from MM:SS.SS to total seconds
        if isinstance(time, str) and ':' in time:
            parts = time.split(':')
            total_seconds = float(parts[0]) * 60 + float(parts[1])
        else:
            total_seconds = float(time)
        
        # For individual events
        if not is_relay:
            record = {
                'name': name_or_team,
                'team': team,
                'event': event_name,
                'best_time': total_seconds,
                'seed_rank': int(rank)
            }
            all_records.append(record)

# Create DataFrame from records
result_df = pd.DataFrame(all_records)

Processing Events: 100%|██████████| 11/11 [00:00<00:00, 108.56it/s]


In [167]:
result_df.sample(5)

,name,team,event,best_time,seed_rank
1191,Calvin Mueller,Hannibal,100 Breaststroke,65.84,76
893,William Newell,Rockhurst,500 Freestyle,311.99,59
1190,Hayes McIntosh,Blue Springs South,100 Breaststroke,65.84,75
195,Langston Adams,Kirkwood,200 Individual Medley,126.36,47
204,Garrison Wells,Staley,200 Individual Medley,127.25,56


In [168]:
# save results
result_df.to_csv('swimmers.csv', index=False)

In [169]:
# split by class
swimmers = pd.read_csv("swimmers.csv")
schools = pd.read_csv("schools.csv")

# Merge Class info into swimmer table
swimmers = swimmers.merge(
    schools[['School', 'Class']],
    left_on='team',
    right_on='School',
    how='left'
).drop(columns=['School'])

In [170]:
swimmers["best_time"] = pd.to_numeric(swimmers["best_time"], errors="coerce")

# Sort by class, then event, then best_time ascending (faster = smaller time)
swimmers = swimmers.sort_values(["Class", "event", "best_time"], ascending=[True, True, True])

swimmers["old_seed_rank"] = swimmers["seed_rank"]

# Now assign new seed ranks within each class+event group
swimmers["seed_rank"] = (
    swimmers.groupby(["Class", "event"])
    .cumcount() + 1
)

# Save or inspect
swimmers.to_csv("swimmers_reseeded.csv", index=False)

In [171]:
class1_swimmers = swimmers[swimmers['Class'] == 1]
class2_swimmers = swimmers[swimmers['Class'] == 2]

In [172]:
swimmers.head(10)

,name,team,event,best_time,seed_rank,Class,old_seed_rank
966,Brody Blatt,Parkway Central,100 Backstroke,50.68,1,1,2
967,Clark Montgomery,Webster Groves,100 Backstroke,51.91,2,1,3
972,Kaiden Cracraft,Notre Dame (Cape Girardeau),100 Backstroke,53.28,3,1,8
973,Burgess Walden,Pembroke Hill,100 Backstroke,54.10,4,1,9
974,Jonathan Grant,Pembroke Hill,100 Backstroke,54.39,5,1,10
975,Mason Wardlow,William Chrisman,100 Backstroke,54.42,6,1,11
977,Cole Strom,Webster Groves,100 Backstroke,54.57,7,1,13
978,Alexander Atkins,Parkway West,100 Backstroke,54.68,8,1,14
980,Calvin Mueller,Hannibal,100 Backstroke,54.89,9,1,16
981,Kolton Evans,Nevada,100 Backstroke,54.99,10,1,17


## 2) Performance model (w/ MCMC!)
I model each swimmer's resulting time using Markov Chain Monte Carlo (MCMC). This best simulates how a swimmer might perform at a specific meet because each swimmer has a performance state *(Good / Average / Bad)* and I add noise of +/- 0.5% or ~0.2s to every outcome.

**To explain the Markov Chain part more:** Swimmers move between these states in their events as the simulated swim meet goes on.
| State   | Time Adjustment      |
| ------- | -------------------- |
| Good    | 0.98 × personal best |
| Average | 1.00 × personal best |
| Bad     | 1.03 × personal best |

These are the transition probabilities of moving between states:
| From → To   | Good | Average | Bad |
| ----------- | ---- | ------- | --- |
| **Good**    | 0.7  | 0.2     | 0.1 |
| **Average** | 0.3  | 0.5     | 0.2 |
| **Bad**     | 0.1  | 0.3     | 0.6 |

Ex. If a swimmer is in the Good state right now, they have:
- a 70% chance to stay Good in the next event,
- a 20% chance to slip to Average,
- a 10% chance to have a Bad race.

Note how it's stochastic, meaning that you don't know the exact outcome but you know the distribution. This is to capture the concept of a swimmer having a 'good' or 'bad' day or race and how that affects other events

**To explain the Monte Carlo part more:** each meet = 1 Monte Carlo trial & a swimmer's performance state will be tested for each meet as well. Each swimmer's time is a random draw from a normal distribution that's centered around their performance state at that time, which should add +/- 0.5% or ~0.2s variability. I will also run the simulation thousands of times to sample it again and again to get a representation of the probabilities & chains.

In [173]:
PERF_STATES = ['Good','Average','Bad']

transition_matrix = {
    'Good': {'Good':0.7,'Average':0.25,'Bad':0.05},
    'Average': {'Good':0.15,'Average':0.7,'Bad':0.15},
    'Bad': {'Good':0.05,'Average':0.25,'Bad':0.7}
}

sigma_mult = {'Good':0.005, 'Average':0.01, 'Bad':0.02}

In [174]:
def step_state(current_state):
    probs = list(transition_matrix[current_state].values())
    next_state = np.random.choice(list(transition_matrix[current_state].keys()), p=probs)
    return next_state

def sample_time(best_time, state):
    sigma = best_time * sigma_mult[state]
    return np.random.normal(best_time, sigma)


## 3) Adaptive interdependence (team momentum)

I also added a simple **team momentum** effect, which is where if earlier events produce unexpectedly good results for a team, later swimmers from the same team also have a small time improvement. This was pretty simple and I can add more sophistication later (fatigue, lane number effects, head-to-head racing, etc.)

In [175]:
def apply_team_momentum(team_scores_by_event, base_time, team, momentum_factor=0.001):
    # team_scores_by_event: dict tracking cumulative unexpected_points (or performance) per team so far
    # base_time: best_time
    # momentum_factor: seconds improvement per unit of team momentum
    momentum = team_scores_by_event.get(team, 0.0)
    adjusted_time = base_time - (momentum * momentum_factor)
    return adjusted_time

## 4) Event Optimization via ILP

Goal: For each simulated performance dataset, assign swimmers to at most 2 events, optimizing total team points.

Use PuLP (or Google OR-Tools) for Integer Linear Programming.

Binary variable x[swimmer, event] = 1 if swimmer enters that event

Constraint: sum(x[swimmer, event]) <= 2

Constraint: exactly 16 swimmers per event (or whatever format)

Objective: maximize total expected team points (from your seed_to_points() logic)

Each Monte Carlo trial gives you a slightly different “optimal meet lineup” based on simulated performances.

Formulate as integer program where decision variables

Relay-specific constraints (exactly 4 per relay lineup).

Event entry limits (e.g., max 2 swimmers per team per event if applicable).

Objective: maximize expected points. But expected points are non-linear (depends on relative entries), so ILP needs approximations (use predicted expected points)

In [176]:
import pulp
import pandas as pd
import numpy as np

POINTS = [20,17,16,15,14,13,12,11,9,7,6,5,4,3,2,1]

def seed_to_points(rank):
    if rank <= 16:
        return POINTS[rank-1]
    else:
        return max(0.5, 10 / rank)

def optimize_event_assignments(df, max_events_per_swimmer=2, swimmers_per_event=16):
    df = df.copy()
    df["expected_points"] = df["seed_rank"].apply(seed_to_points)
    
    swimmers = df["name"].unique()
    events = df["event"].unique()
    
    # Define model
    model = pulp.LpProblem("SwimMeetOptimization", pulp.LpMaximize)
    
    # Decision variables
    x = pulp.LpVariable.dicts(
        "x",
        ((r["name"], r["event"]) for _, r in df.iterrows()),
        cat="Binary"
    )
    
    # Objective: maximize expected points
    model += pulp.lpSum([
        r["expected_points"] * x[(r["name"], r["event"])] for _, r in df.iterrows()
    ])
    
    # Constraint: each swimmer ≤ 2 events
    for s in swimmers:
        model += pulp.lpSum([
            x[(r["name"], r["event"])] for _, r in df[df["name"] == s].iterrows()
        ]) <= max_events_per_swimmer
    
    # Constraint: each event has 16 swimmers if possible
    for e in events:
        event_swimmers = df[df["event"] == e]
        if len(event_swimmers) >= swimmers_per_event:
            model += pulp.lpSum([
                x[(r["name"], e)] for _, r in event_swimmers.iterrows()
            ]) == swimmers_per_event
        else:
            model += pulp.lpSum([
                x[(r["name"], e)] for _, r in event_swimmers.iterrows()
            ]) <= len(event_swimmers)
    
    # Mutually exclusive event pairs
    conflict_pairs = [
        ("200 Freestyle", "200 IM"),
        ("50 Freestyle", "200 IM"),
        ("100 Freestyle", "100 Butterfly"),
        ("100 Backstroke", "100 Breaststroke"),
        ("500 Freestyle", "100 Freestyle"),
    ]
    
    for s in swimmers:
        swimmer_events = df[df["name"] == s]["event"].unique()
        for e1, e2 in conflict_pairs:
            if e1 in swimmer_events and e2 in swimmer_events:
                model += x[(s, e1)] + x[(s, e2)] <= 1
    
    # Solve
    model.solve(pulp.PULP_CBC_CMD(msg=False))
    
    # Collect chosen
    selected_rows = []
    for _, r in df.iterrows():
        if pulp.value(x[(r["name"], r["event"])]) > 0.5:
            selected_rows.append(r)
    selected = pd.DataFrame(selected_rows)
    
    # Reseed
    selected["new_seed_rank"] = (
        selected.groupby("event")["best_time"].rank("dense").astype(int)
    )
    selected = selected.sort_values(["event", "new_seed_rank"]).reset_index(drop=True)
    
    return selected

In [177]:
output = optimize_event_assignments(class1_swimmers)
output.to_csv("class1_assignments.csv")

## 4) Scoring and running a single simulated meet

I score the meet using top 16 scoring points: [20,17,16,15,14,13,12,11,9,7,6,5,4,3,2,1]. I'll add Relay functionality later

In [101]:
POINTS = [20,17,16,15,14,13,12,11,9,7,6,5,4,3,2,1]  # assign to places 1..16

def simulate_one_meet(df, assignments, verbose=False):
    # df: DataFrame rows: name, team, event, best_time
    # assignments: dict mapping swimmer -> list of events they're entered (enforces <=2 events)
    # returns: dataframe of results and team scores
    
    # initialize states for each swimmer randomly (or all Average)
    swimmer_states = {name: np.random.choice(PERF_STATES, p=[0.2,0.6,0.2]) for name in df['name'].unique()}
    
    results = []
    # track team 'unexpectedness' as sum of (expected_rank - actual_rank) positive values
    team_momentum = {}
    
    # simulate per event in a deterministic order (you can randomize event order to capture ordering effects)
    events_order = df['event'].unique().tolist()
    for event in events_order:
        # swimmers entered in this event
        entrants = [s for s, evs in assignments.items() if event in evs]
        rows = df[df['name'].isin(entrants) & (df['event']==event)]
        
        sim_rows = []
        for _, r in rows.iterrows():
            name = r['name']; team = r['team']; base = r['best_time']
            # step state to model carryover between events for the swimmer
            cur_state = swimmer_states[name]
            next_state = step_state(cur_state)
            swimmer_states[name] = next_state
            # apply team momentum to base time
            adj_base = apply_team_momentum(team_momentum, base, team)
            time = sample_time(adj_base, next_state)
            sim_rows.append({'name':name,'team':team,'event':event,'time':time,'state':next_state})
        
        # rank by time and assign points
        if not sim_rows:
            continue
        sim_df = pd.DataFrame(sim_rows).sort_values('time').reset_index(drop=True)
        sim_df['place'] = sim_df.index + 1
        sim_df['points'] = sim_df['place'].apply(lambda p: POINTS[p-1] if p<=len(POINTS) else 0)
        
        # update team momentum: if a swimmer beats expectation (seed_rank) by a margin, increase momentum
        # expectation: use seed_rank from df for that swimmer/event; if not present, assume average.
        for i, row in sim_df.iterrows():
            name = row['name']; team = row['team']; place = row['place']
            seed_row = df[(df['name']==name)&(df['event']==event)]
            if not seed_row.empty:
                seed_rank = int(seed_row['seed_rank'].iloc[0])
                unexpected = max(0, seed_rank - place)
                team_momentum[team] = team_momentum.get(team, 0.0) + unexpected * 0.2  # tuning parameter
        results.append(sim_df)
    
    full = pd.concat(results, ignore_index=True) if results else pd.DataFrame(columns=['name','team','event','time','place','points'])
    team_scores = full.groupby('team')['points'].sum().to_dict()
    return full, team_scores

## 5) Monte Carlo simulation to estimate probabilities

Run many simulations and compute distributions for team scores, win probabilities, and event medal probabilities.


In [102]:
def run_monte_carlo(df, assignments, n_sims=2000):
    team_scores_all = defaultdict(list)
    
    for i in tqdm(range(n_sims), desc="Running Monte Carlo simulations"):
        _, team_scores = simulate_one_meet(df, assignments)
        
        for team, score in team_scores.items():
            team_scores_all[team].append(score)
    
    return team_scores_all

In [103]:
# Example naive assignments: allow each swimmer up to 2 events; we assign the events they are seeded in the df
assignments = {}
for name in result_df['name'].unique():
    events_for = result_df[result_df['name']==name]['event'].tolist()
    assignments[name] = events_for[:2]  # take up to first 2 seeds for example

In [104]:
team_scores_all = run_monte_carlo(result_df, assignments, n_sims=500)

Running Monte Carlo simulations: 100%|██████████| 500/500 [01:45<00:00,  4.72it/s]


{'Battle': 6.626,
 'Blue Springs': 10.544,
 'Blue Springs South': 16.494,
 'Camdenton': 0.0,
 'Capital City': 9.538,
 'Carthage': 0.356,
 'Central (Cape Girardeau)': 0.0,
 'Central (Springfield)': 11.004,
 'Central (St. Joseph)': 7.34,
 'Chaminade College Prep': 117.95,
 'Christian Brothers College': 8.484,
 'Clayton': 3.674,
 'Columbia Independent': 7.02,
 'De Smet Jesuit': 26.166,
 'Eureka': 7.358,
 'Farmington': 2.544,
 'Francis Howell': 0.132,
 'Francis Howell Central': 0.0,
 'Francis Howell North': 0.076,
 'Ft. Zumwalt North': 0.004,
 'Ft. Zumwalt South': 11.018,
 'Glendale': 24.138,
 'Grain Valley': 0.002,
 'Hannibal': 0.556,
 'Hickman': 27.112,
 'Hillcrest': 0.06,
 'Holt': 17.332,
 'Jackson': 0.024,
 'John Burroughs': 9.044,
 'Kearney': 34.988,
 'Kickapoo': 13.004,
 'Kirkwood': 74.822,
 'Ladue Horton Watkins': 1.63,
 'Lafayette (Wildwood)': 15.296,
 'Lebanon': 1.09,
 "Lee's Summit": 27.72,
 "Lee's Summit North": 26.098,
 "Lee's Summit West": 3.136,
 'Liberty': 46.954,
 'Liberty 

In [105]:
# show simple mean scores
{team: np.mean(scores) for team,scores in team_scores_all.items()}

{'Battle': 6.626,
 'Blue Springs': 10.544,
 'Blue Springs South': 16.494,
 'Camdenton': 0.0,
 'Capital City': 9.538,
 'Carthage': 0.356,
 'Central (Cape Girardeau)': 0.0,
 'Central (Springfield)': 11.004,
 'Central (St. Joseph)': 7.34,
 'Chaminade College Prep': 117.95,
 'Christian Brothers College': 8.484,
 'Clayton': 3.674,
 'Columbia Independent': 7.02,
 'De Smet Jesuit': 26.166,
 'Eureka': 7.358,
 'Farmington': 2.544,
 'Francis Howell': 0.132,
 'Francis Howell Central': 0.0,
 'Francis Howell North': 0.076,
 'Ft. Zumwalt North': 0.004,
 'Ft. Zumwalt South': 11.018,
 'Glendale': 24.138,
 'Grain Valley': 0.002,
 'Hannibal': 0.556,
 'Hickman': 27.112,
 'Hillcrest': 0.06,
 'Holt': 17.332,
 'Jackson': 0.024,
 'John Burroughs': 9.044,
 'Kearney': 34.988,
 'Kickapoo': 13.004,
 'Kirkwood': 74.822,
 'Ladue Horton Watkins': 1.63,
 'Lafayette (Wildwood)': 15.296,
 'Lebanon': 1.09,
 "Lee's Summit": 27.72,
 "Lee's Summit North": 26.098,
 "Lee's Summit West": 3.136,
 'Liberty': 46.954,
 'Liberty 

## score the meet

## visualize all teh different outcomes

In [ ]:
# Assume df_results has one row per simulation per team
team_totals = df_results.groupby(['simulation_id', 'team'])['points'].sum().reset_index()
winners = team_totals.loc[team_totals.groupby('simulation_id')['points'].idxmax()]
win_probs = winners['team'].value_counts(normalize=True).sort_values()

plt.figure(figsize=(10,5))
win_probs.plot(kind='barh', color='skyblue')
plt.title("🏆 Probability of Winning the State Meet")
plt.xlabel("Win Probability")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.kdeplot(data=team_totals, x='points', hue='team', fill=True, alpha=0.3)
plt.title("Team Points Distribution Across Simulated Meets")
plt.xlabel("Total Points")
plt.ylabel("Density")
plt.show()

In [ ]:
medal_probs = (
    df_results[df_results['place'] <= 3]
    .groupby(['swimmer_name', 'event', 'place'])
    .size()
    .groupby(level=[0,1])
    .apply(lambda x: x / x.sum())
    .reset_index(name='probability')
)

plt.figure(figsize=(12,6))
sns.barplot(data=medal_probs, x='swimmer_name', y='probability', hue='place')
plt.title("🥇🥈🥉 Medal Probabilities by Swimmer and Event")
plt.xticks(rotation=90)
plt.ylabel("Probability of Podium Finish")
plt.tight_layout()
plt.show()

In [ ]:
event_strength = (
    df_results[df_results['place'] == 1]
    .groupby(['event', 'team'])
    .size()
    .unstack(fill_value=0)
)

plt.figure(figsize=(10,7))
sns.heatmap(event_strength, cmap="YlGnBu", annot=True, fmt='d')
plt.title("Event Dominance: # of Simulated Wins by Team")
plt.xlabel("Team")
plt.ylabel("Event")
plt.show()

In [ ]:
# Suppose you ran the optimizer many times and stored best assignments
assignments = []
for run_id, best in enumerate(best_assignments_list):
    for swimmer, events in best.items():
        for e in events:
            assignments.append((run_id, swimmer, e))
df_assign = pd.DataFrame(assignments, columns=["run", "swimmer", "event"])

opt_counts = df_assign.groupby(['swimmer', 'event']).size().unstack(fill_value=0)

plt.figure(figsize=(12,7))
sns.heatmap(opt_counts, cmap="magma", annot=True, fmt='d')
plt.title("Most Frequently Selected Lineup Assignments Across Optimizations")
plt.xlabel("Event")
plt.ylabel("Swimmer")
plt.show()

## Final notes and next steps

This notebook is intentionally modular so you can:

- Tune `sigma_mult`, `transition_matrix`, and `team_momentum` to better fit historical variability.
- Implement relay-specific rules (4 swimmers per relay, relay scoring, relay fatigue).
- Improve event-order modeling (randomize event order per sim to capture early-late effects).
- Use a stronger optimizer (genetic algorithm, integer programming) for event assignment.

If you'd like, I can now:
1. Run this notebook on a realistic extract of your spreadsheet (if you paste CSV text or upload the file).
2. Add relay handling + lane constraints.
3. Add visualizations of distributions and per-swimmer expected points. ******

Tell me which of the above you'd like next, or upload your CSV and I'll run the simulations on your real data.
